<a href="https://www.kaggle.com/code/ainurrohmanbwx/sales-analytics?scriptVersionId=163661094" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Executive Summary

From the results of the analysis of online shop transactions in 2019, there are important points:

1. The highest sales transactions are in December.
2. The city with the most transactions is the city of San Francisco, namely 24.1%.
3. There are 5 best selling items, namely:
     - AA Batteries (4-pack)
     - AAA Batteries (4-pack)
     - Lightning Charging Cable
     - USB-C Charging Cable
     - Wired Headphones
4. There are 3 types of transactions with items that are often purchased simultaneously:
     - iPhone, Lightning Charging Cable, and Wired Headphones.
     - Google phone, USB-C Charging Cable, and Wired Headphones.
     - Vareebadd phone and Wired Headphones.

In [1]:
# Disable warning

import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=FutureWarning)

# Load Data

In [2]:
import os
import pandas as pd

all_files = os.listdir('/kaggle/input/practice-eda-on-this-sales-dataset')
all_files

['Sales_August_2019.csv',
 'Sales_May_2019.csv',
 'Sales_February_2019.csv',
 'Sales_November_2019.csv',
 'Sales_January_2019.csv',
 'Sales_March_2019.csv',
 'Sales_September_2019.csv',
 'Sales_April_2019.csv',
 'Sales_July_2019.csv',
 'Sales_October_2019.csv',
 'Sales_June_2019.csv',
 'Sales_December_2019.csv']

In [3]:
all_data = []

for file in all_files:
    file_path = '/kaggle/input/practice-eda-on-this-sales-dataset/' + file
    df = pd.read_csv(file_path)
    all_data.append(df)

all_data = pd.concat(all_data, ignore_index=True)

In [4]:
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,236670,Wired Headphones,2,11.99,08/31/19 22:21,"359 Spruce St, Seattle, WA 98101"
1,236671,Bose SoundSport Headphones,1,99.99,08/15/19 15:11,"492 Ridge St, Dallas, TX 75001"
2,236672,iPhone,1,700.0,08/06/19 14:40,"149 7th St, Portland, OR 97035"
3,236673,AA Batteries (4-pack),2,3.84,08/29/19 20:59,"631 2nd St, Los Angeles, CA 90001"
4,236674,AA Batteries (4-pack),2,3.84,08/15/19 19:53,"736 14th St, New York City, NY 10001"


In [5]:
all_data.tail()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
186845,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016"
186846,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001"
186847,319668,Vareebadd Phone,1,400,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101"
186848,319669,Wired Headphones,1,11.99,12/03/19 10:39,"778 River St, Dallas, TX 75001"
186849,319670,Bose SoundSport Headphones,1,99.99,12/21/19 21:45,"747 Chestnut St, Los Angeles, CA 90001"


In [6]:
all_data.shape

(186850, 6)

In [7]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


# Data Preprocessing

### Are there any missing values?

In [8]:
# Check for missing values
missing_values = all_data.isnull().sum()

# Display columns with missing values and the count of missing values
missing_values = missing_values[missing_values > 0]

if not missing_values.empty:
    print("Columns with missing values:")
    for column, count in missing_values.items():
        print(f"{column}: {count} missing values")
else:
    print("There are no columns with missing value")

Columns with missing values:
Order ID: 545 missing values
Product: 545 missing values
Quantity Ordered: 545 missing values
Price Each: 545 missing values
Order Date: 545 missing values
Purchase Address: 545 missing values


In [9]:
# remove missing values
all_data = all_data.dropna()

In [10]:
#check again after removing
missing_values = all_data.isnull().sum()

# Display columns with missing values and the count of missing values
missing_values = missing_values[missing_values > 0]

if not missing_values.empty:
    print("Columns with missing values:")
    for column, count in missing_values.items():
        print(f"{column}: {count} missing values")
else:
    print("There are no columns with missing value")

There are no columns with missing value


### Is there any duplicate data?

In [11]:
if all_data.duplicated().any():
    print(f"There are as many as {all_data.duplicated().sum()} duplicate data.")
else:
    print("There are no duplicate data.")

There are as many as 618 duplicate data.


In [12]:
# remove duplicate rows
all_data = all_data.drop_duplicates()

In [13]:
# Check again after removing
if all_data.duplicated().any():
    print(f"There are as many as {all_data.duplicated().sum()} duplicate data.")
else:
    print("There are no duplicate data.")

There are no duplicate data.


In [14]:
all_data.shape

(185687, 6)

# Analyze

### What is the most successful month for sales?

In this question, we use three features, namely: 'Quantity Ordered', 'Price Each', and 'Order Date'.
- Order Quantity and Price Each are used to see how many sales.
- Order Date is used to indicate the month in which sales were highest

We extract these three features so that we can answer our questions. To see which month has the most sales, we use a bar chart.

In [15]:
all_data[['Quantity Ordered', 'Price Each', 'Order Date']].dtypes

Quantity Ordered    object
Price Each          object
Order Date          object
dtype: object

In [16]:
# Extract 'Month' from 'Order date'
all_data['Month'] = all_data['Order Date'].apply(lambda x: x.split('/')[0])
all_data['Month'].head()

0    08
1    08
2    08
3    08
4    08
Name: Month, dtype: object

In [17]:
# Ensure data is in the form of numbers
all_data['Month'].unique()

array(['08', 'Order Date', '09', '05', '06', '02', '03', '11', '12', '01',
       '04', '10', '07'], dtype=object)

In [18]:
# Remove 'order date' data
filter_month = all_data['Month'] == 'Order Date'

all_data = all_data[~filter_month]

In [19]:
# Customize the data type of each feature

all_data['Month'] = all_data['Month'].astype(int)
all_data['Quantity Ordered'] = all_data['Quantity Ordered'].astype(int)
all_data['Price Each'] = all_data['Price Each'].astype(float)

In [20]:
all_data[['Quantity Ordered', 'Price Each', 'Order Date']].dtypes

Quantity Ordered      int64
Price Each          float64
Order Date           object
dtype: object

In [21]:
# Viewing 'Sales' by creating the 'Sales' feature
all_data['Sales'] = all_data['Quantity Ordered'] * all_data['Price Each']
all_data['Sales'].head()

0     23.98
1     99.99
2    700.00
3      7.68
4      7.68
Name: Sales, dtype: float64

In [22]:
import plotly.graph_objs as go

# Group by month and sum the sales
monthly_sales = all_data.groupby(['Month'])['Sales'].sum()

# Create a line plot trace
trace = go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales.values,
    mode='lines+markers',
    marker=dict(color='blue'), 
    name='Monthly Sales'
)

# Create layout
layout = go.Layout(
    title='Monthly Sales',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Sales'),
    hovermode='closest',
)

# Create figure
fig = go.Figure(data=[trace], layout=layout)

# Show plot
fig.show()

**Highlight**:

In 2019, the highest sales were in December.

### Which city has the highest number of orders?

To see the distribution of the most sales in each city, we use the "'Purchase Address'" feature which contains user purchase addresses. We extract this feature and just take the city. After that we count how many cities there are. To display it, use a pie chart.

In [23]:
all_data['Purchase Address'].head()

0        359 Spruce St, Seattle, WA 98101
1          492 Ridge St, Dallas, TX 75001
2          149 7th St, Portland, OR 97035
3       631 2nd St, Los Angeles, CA 90001
4    736 14th St, New York City, NY 10001
Name: Purchase Address, dtype: object

In [24]:
all_data['City'] = all_data['Purchase Address'].str.split(',').str.get(1)
all_data['City'].head()

0           Seattle
1            Dallas
2          Portland
3       Los Angeles
4     New York City
Name: City, dtype: object

In [25]:
fig = go.Figure(data=[go.Pie(labels=all_data['City'].value_counts().index, values=all_data['City'].value_counts().values)])

# Update layout for better appearance
fig.update_layout(
    title='Distribution of Cities',
)

# Show the plot
fig.show()

**Highlight**

The largest sales distribution in 2019 was in the city of San Francisco. With a total of 44,662 transactions or 24.1% of the population. Followed by the city of Los Angeles with 29,564 or 15.9% of the population and the city of New York with 24,847 or 13.4% of the population.

### Which product had the highest sales volume, and what factors contributed to its popularity?

To see which items have the highest sales, we use the "Quantity Ordered" feature, we add up the transactions for each item. Then we display it using a bar chart.

To see what factors cause high sales, we use the "Price Each" feature. We look at the average price for the number of transactions for each item. After that we display it using a scatter plot.

We combine these two charts (bar and scatter) and then analyze the relationship between item sales and their average price.To see which items have the highest sales, we use the "Quantity Ordered" feature, we add up the transactions for each item. Then we display it using a bar chart.

In [26]:
order_count = all_data.groupby(['Product']).agg({'Quantity Ordered':'sum', 'Price Each':'mean'}).reset_index()
order_count.head()

,Product,Quantity Ordered,Price Each
0,20in Monitor,4126,109.99
1,27in 4K Gaming Monitor,6239,389.99
2,27in FHD Monitor,7541,149.99
3,34in Ultrawide Monitor,6192,379.99
4,AA Batteries (4-pack),27615,3.84


In [27]:
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar plot for Quantity Ordered
fig.add_trace(go.Bar(x=order_count['Product'], y=order_count['Quantity Ordered'], name='Quantity Ordered'), secondary_y=False)

# Add line plot for Price Each
fig.add_trace(go.Scatter(x=order_count['Product'], y=order_count['Price Each'], mode='lines', name='Price Each'), secondary_y=True)

# Update layout
fig.update_layout(
    title='Quantity Ordered and Price Each for Products',
    xaxis=dict(title='Product'),
    yaxis=dict(title='Quantity Ordered', rangemode='tozero'),
    yaxis2=dict(title='Price Each', overlaying='y', side='right'),
)

# Show the plot
fig.show()

**Data**

The sales with the most transactions are AAA Batteries, followed by AA Batteries.

**Insights**

From the two items with the most transactions, we can conclude that people have many electronic devices that are powered by batteries, such as: Remote controls, audio devices, toys, etc.

**Action**

The company stocks various types of batteries, not just AAA and AA types. Because electronic devices that use batteries currently use various types of batteries, such as: C and D, 9V batteries, lithium batteries, and rechargeable batteries.

### What is the trend of the most sold product over time?

In this analysis, we use two features, namely: "Products" and "Months". The product feature is used to see the 5 products with the most transactions, while the months feature is to see trends over time.In this analysis, we use two features, namely: "Products" and "Months". The product feature is used to see the 5 products with the most transactions, while the months feature is to see trends over time.

In [28]:
all_data['Product'].value_counts().index[:5]

Index(['USB-C Charging Cable', 'Lightning Charging Cable',
       'AAA Batteries (4-pack)', 'AA Batteries (4-pack)', 'Wired Headphones'],
      dtype='object', name='Product')

In [29]:
most_sold_product = all_data['Product'].value_counts().index[:5]
most_sold_product

Index(['USB-C Charging Cable', 'Lightning Charging Cable',
       'AAA Batteries (4-pack)', 'AA Batteries (4-pack)', 'Wired Headphones'],
      dtype='object', name='Product')

In [30]:
most_sold_products_data = all_data[all_data['Product'].isin(most_sold_product)]
most_sold_products_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City
0,236670,Wired Headphones,2,11.99,08/31/19 22:21,"359 Spruce St, Seattle, WA 98101",8,23.98,Seattle
3,236673,AA Batteries (4-pack),2,3.84,08/29/19 20:59,"631 2nd St, Los Angeles, CA 90001",8,7.68,Los Angeles
4,236674,AA Batteries (4-pack),2,3.84,08/15/19 19:53,"736 14th St, New York City, NY 10001",8,7.68,New York City
5,236675,Wired Headphones,1,11.99,08/02/19 23:54,"470 Hill St, San Francisco, CA 94016",8,11.99,San Francisco
8,236678,Wired Headphones,1,11.99,08/25/19 20:11,"58 9th St, San Francisco, CA 94016",8,11.99,San Francisco


In [31]:
most_sold_products_data.groupby(['Month','Product']).size()

Month  Product                 
1      AA Batteries (4-pack)       1037
       AAA Batteries (4-pack)      1084
       Lightning Charging Cable    1069
       USB-C Charging Cable        1171
       Wired Headphones            1004
2      AA Batteries (4-pack)       1274
       AAA Batteries (4-pack)      1320
       Lightning Charging Cable    1393
       USB-C Charging Cable        1511
       Wired Headphones            1179
3      AA Batteries (4-pack)       1672
       AAA Batteries (4-pack)      1645
       Lightning Charging Cable    1749
       USB-C Charging Cable        1766
       Wired Headphones            1512
4      AA Batteries (4-pack)       2062
       AAA Batteries (4-pack)      1988
       Lightning Charging Cable    2197
       USB-C Charging Cable        2074
       Wired Headphones            1888
5      AA Batteries (4-pack)       1821
       AAA Batteries (4-pack)      1888
       Lightning Charging Cable    1929
       USB-C Charging Cable        1879
       W

In [32]:
most_sold_products_data.groupby(['Month','Product']).size().unstack()

Product,AA Batteries (4-pack),AAA Batteries (4-pack),Lightning Charging Cable,USB-C Charging Cable,Wired Headphones
Month,,,,,
1,1037,1084,1069,1171,1004
2,1274,1320,1393,1511,1179
3,1672,1645,1749,1766,1512
4,2062,1988,2197,2074,1888
5,1821,1888,1929,1879,1729
6,1540,1451,1560,1531,1334
7,1555,1554,1690,1667,1434
8,1357,1340,1354,1339,1191
9,1314,1281,1324,1451,1173


In [33]:
grouped_data = most_sold_products_data.groupby(['Month', 'Product']).size().unstack()

# Create traces for each product
traces = []
for column in grouped_data.columns:
    trace = go.Scatter(
        x=grouped_data.index,
        y=grouped_data[column],
        mode='lines+markers',
        name=column
    )
    traces.append(trace)

# Create layout
layout = go.Layout(
    title='Most Sold Products Over Time',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Number of Sales'),
    hovermode='closest',
)

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show plot
fig.show()

**Highlight**

Of the 5 best-selling products, these 5 products are unique, namely trends that are almost the same every month.

### Which products are frequently sold together?

In analyzing which items are frequently purchased together, we use the "Order ID" and "Products" features.

In [34]:
duplicated_data = all_data[all_data['Order ID'].duplicated(keep=False)]
duplicated_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City
46,236716,AA Batteries (4-pack),2,3.84,08/17/19 20:12,"84 Cedar St, San Francisco, CA 94016",8,7.68,San Francisco
47,236716,USB-C Charging Cable,1,11.95,08/17/19 20:12,"84 Cedar St, San Francisco, CA 94016",8,11.95,San Francisco
60,236729,iPhone,1,700.00,08/09/19 09:28,"635 Lake St, San Francisco, CA 94016",8,700.00,San Francisco
61,236729,Apple Airpods Headphones,1,150.00,08/09/19 09:28,"635 Lake St, San Francisco, CA 94016",8,150.00,San Francisco
62,236730,Google Phone,1,600.00,08/02/19 10:17,"445 Lincoln St, New York City, NY 10001",8,600.00,New York City


In [35]:
duplicated_data.shape

(14128, 9)

In [36]:
grouped_products = duplicated_data.groupby(['Order ID'])['Product'].apply(lambda x: ','.join(x)).reset_index().rename(columns={'Product':'Grouped Products'})
grouped_products.head()

,Order ID,Grouped Products
0,141275,"USB-C Charging Cable,Wired Headphones"
1,141290,"Apple Airpods Headphones,AA Batteries (4-pack)"
2,141365,"Vareebadd Phone,Wired Headphones"
3,141384,"Google Phone,USB-C Charging Cable"
4,141450,"Google Phone,Bose SoundSport Headphones"


In [37]:
grouped_products.shape

(6879, 2)

In [38]:
duplicate_products = duplicated_data.merge(grouped_products, how='left', on='Order ID')
duplicate_products.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Grouped Products
0,236716,AA Batteries (4-pack),2,3.84,08/17/19 20:12,"84 Cedar St, San Francisco, CA 94016",8,7.68,San Francisco,"AA Batteries (4-pack),USB-C Charging Cable"
1,236716,USB-C Charging Cable,1,11.95,08/17/19 20:12,"84 Cedar St, San Francisco, CA 94016",8,11.95,San Francisco,"AA Batteries (4-pack),USB-C Charging Cable"
2,236729,iPhone,1,700.00,08/09/19 09:28,"635 Lake St, San Francisco, CA 94016",8,700.00,San Francisco,"iPhone,Apple Airpods Headphones"
3,236729,Apple Airpods Headphones,1,150.00,08/09/19 09:28,"635 Lake St, San Francisco, CA 94016",8,150.00,San Francisco,"iPhone,Apple Airpods Headphones"
4,236730,Google Phone,1,600.00,08/02/19 10:17,"445 Lincoln St, New York City, NY 10001",8,600.00,New York City,"Google Phone,Wired Headphones"


In [39]:
duplicate_products.shape

(14128, 10)

In [40]:
drop_duplicate_order_id = duplicate_products.drop_duplicates(subset=['Order ID'])
drop_duplicate_order_id.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Grouped Products
0,236716,AA Batteries (4-pack),2,3.84,08/17/19 20:12,"84 Cedar St, San Francisco, CA 94016",8,7.68,San Francisco,"AA Batteries (4-pack),USB-C Charging Cable"
2,236729,iPhone,1,700.00,08/09/19 09:28,"635 Lake St, San Francisco, CA 94016",8,700.00,San Francisco,"iPhone,Apple Airpods Headphones"
4,236730,Google Phone,1,600.00,08/02/19 10:17,"445 Lincoln St, New York City, NY 10001",8,600.00,New York City,"Google Phone,Wired Headphones"
6,236799,Google Phone,1,600.00,08/01/19 12:38,"309 2nd St, San Francisco, CA 94016",8,600.00,San Francisco,"Google Phone,Wired Headphones"
8,236829,Vareebadd Phone,1,400.00,08/16/19 08:51,"27 2nd St, Los Angeles, CA 90001",8,400.00,Los Angeles,"Vareebadd Phone,USB-C Charging Cable"


In [41]:
drop_duplicate_order_id.shape

(6879, 10)

In [42]:
# Get the top 5 grouped products
top_products = drop_duplicate_order_id['Grouped Products'].value_counts()[:5]

# Create pie chart trace
trace = go.Pie(
    labels=top_products.index,
    values=top_products.values,
    hole=0.3,
)

# Create layout
layout = go.Layout(
    title='Top 5 Grouped Products Distribution'
)

# Create figure
fig = go.Figure(data=[trace], layout=layout)

# Show plot
fig.show()

**Data**

Items that are often purchased together are:

1. iPhone. Often buyers when buying an iPhone also buy a lightning charging cable and wired headphones.
2. Google phone. Often buyers when buying a Google phone also buy a USB-C charging cable and wired headphones.
3. Vareebadd phone. Often buyers when buying a Vareebadd phone also buy a USB-C charging cable.

**Insight**

Of these 3 devices (iPhone, Google phone, vareebadd phone) buyers always buy charging cables and headphones too. This is because this device does not provide these accessories.

**Action**

Based on the data and insight provided, online stores must provide a variety of smartphone accessories so that buyers can complete their smartphone needs.